In [4]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [7]:
data_path = "GuavaDiseaseDataset"
os.listdir(data_path)

['train', 'val', 'test']

In [8]:
transforms = transforms.Compose([
    transforms.Resize((224, 224, )),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(root=os.path.join(data_path, "train"), transform=transforms)
val_dataset = ImageFolder(root=os.path.join(data_path, "val"), transform=transforms)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [11]:
model = models.resnet18(pretrained=True)
num_classes = len(train_dataset.classes)
#model
model.fc = nn.Linear(model.fc.in_features, num_classes)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [12]:
device

'cuda'

In [13]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(model, train_dataloader, val_dataloader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_dataloader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}')
        
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_accuracy = 100 * correct / total # in percentage
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {val_accuracy:.4f} %, Validation Loss: {val_loss / len(val_dataloader.dataset):.4f}')


In [15]:
train_model(model, train_dataloader, val_dataloader, criterion, optimizer, num_epochs=10)

Epoch 1/10, Training Loss: 0.1418
Epoch 1/10, Validation Accuracy: 97.7483 %, Validation Loss: 0.0753
Epoch 2/10, Training Loss: 0.1064
Epoch 2/10, Validation Accuracy: 96.6887 %, Validation Loss: 0.0929
Epoch 3/10, Training Loss: 0.0316
Epoch 3/10, Validation Accuracy: 91.5232 %, Validation Loss: 0.2913
Epoch 4/10, Training Loss: 0.0374
Epoch 4/10, Validation Accuracy: 94.1722 %, Validation Loss: 0.1602
Epoch 5/10, Training Loss: 0.1149
Epoch 5/10, Validation Accuracy: 96.8212 %, Validation Loss: 0.0866
Epoch 6/10, Training Loss: 0.0635
Epoch 6/10, Validation Accuracy: 97.6159 %, Validation Loss: 0.0651
Epoch 7/10, Training Loss: 0.0215
Epoch 7/10, Validation Accuracy: 99.4702 %, Validation Loss: 0.0170
Epoch 8/10, Training Loss: 0.0084
Epoch 8/10, Validation Accuracy: 99.2053 %, Validation Loss: 0.0172
Epoch 9/10, Training Loss: 0.0090
Epoch 9/10, Validation Accuracy: 99.4702 %, Validation Loss: 0.0140
Epoch 10/10, Training Loss: 0.0075
Epoch 10/10, Validation Accuracy: 99.6026 %, Va

In [16]:
from PIL import Image

def predict_image(model, image_path, transform):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    return train_dataset.classes[predicted.item()]

In [20]:
#img = "GuavaDiseaseDataset/test/fruit_fly/9_unsharp_clahe_augmented_6.png"
img = "GuavaDiseaseDataset/test/healthy_guava/38_unsharp_clahe_augmented_3.png"
predict_image(model, img, transforms)

'healthy_guava'